In [8]:
import torch
import numpy as np

In [9]:
def myConv3d(
    input_image,
    in_channels,
    out_channels,
    kernel_size,
    stride=1,
    padding=0,
    dilation=1,
    groups=1,
    bias=True,
    padding_mode='zeros',
    ):
    if (in_channels%groups != 0) or (out_channels%groups!=0):
      raise Exception('in_channels and out_channels must be divisible by groups')
    if bias == True:
      bias_val = torch.rand(out_channels)
    else:
      bias_val = torch.zeros(out_channels)
    if (padding_mode == 'zeros'):
      pad = torch.nn.ZeroPad3d(padding)
      input_image = pad(input_image)
    if (padding_mode == 'reflect'):
      pad = torch.nn.ReflectionPad3d(padding)
      input_image = pad(input_image)
    if (padding_mode == 'replicate'):
      pad = torch.nn.ReplicationPad3d(padding)
      input_image = pad(input_image)
    if (padding_mode == 'circular'):
      pad = torch.nn.CircularPad3d(padding)
      input_image = pad(input_image)
    if type(kernel_size) == tuple:
      weights = torch.rand(
          out_channels,
          in_channels//groups,
          kernel_size[0],
          kernel_size[1],
          kernel_size[2],
          )
    if type(kernel_size) == int:
      weights = torch.rand(
          out_channels,
          in_channels//groups,
          kernel_size,
          kernel_size,
          kernel_size,
          )
    result = []
    for l in range(out_channels):
      ceil_of_res = np.array([])
      for k in range (0, input_image.shape[1]-((weights.shape[2]-1)*dilation+1)+1, stride):
        for i in range (0, input_image.shape[2]-((weights.shape[3]-1)*dilation+1)+1, stride):
          for j in range (0, input_image.shape[3]-((weights.shape[4]-1)*dilation+1)+1, stride):
            preres = 0
            for c in range (in_channels//groups):
              if groups>1:
                val = input_image[l*(in_channels//groups)+c][
                    k:k+(weights.shape[2]-1)*dilation+1:dilation,
                    i:i+(weights.shape[3]-1)*dilation+1:dilation,
                    j:j+(weights.shape[4]-1)*dilation+1:dilation,
                    ]
              else:
                val = input_image[c][
                    k:k+(weights.shape[2]-1)*dilation+1:dilation,
                    i:i+(weights.shape[3]-1)*dilation+1:dilation,
                    j:j+(weights.shape[4]-1)*dilation+1:dilation,
                    ]
              prepreres = (val*weights[l][c]).sum()
              preres = preres + prepreres
            ceil_of_res = np.append(ceil_of_res, float(preres + bias_val[l]))

      result.append(
          ceil_of_res.reshape(
              (input_image.shape[1]-((weights.shape[2]-1)*dilation+1))//stride+1,
              (input_image.shape[2]-((weights.shape[3]-1)*dilation+1))//stride+1,
              (input_image.shape[3]-((weights.shape[4]-1)*dilation+1))//stride+1,
              ))

    return np.array(result), torch.tensor(np.array(weights)), torch.tensor(np.array(bias_val))

In [10]:
tensor1 = torch.rand(4, 6, 6, 6)
tensor2 = torch.rand(3, 24, 28, 20)
tensor3 = torch.rand(1, 1, 1, 1)

In [11]:
result, kernel, bias_val = myConv3d(
    tensor1,
    in_channels=4,
    out_channels=2,
    kernel_size=3,
    stride=1,
    padding=0,
    dilation=1,
    groups=2,
    bias=True,
    padding_mode='zeros',
    )
torchFunction = torch.nn.Conv3d(
    in_channels=4,
    out_channels=2,
    kernel_size=3,
    stride=1,
    padding=0,
    dilation=1,
    groups=2,
    bias=True,
    padding_mode='zeros',
    )
torchFunction.weight.data = kernel
torchFunction.bias.data = bias_val
myResult = str(np.round(result, 2))
torchResult = str(np.round(np.array(torchFunction(tensor1).data), 2))
torchResult == myResult

True

In [12]:
result2, kernel2, bias_val2 = myConv3d(
    tensor2,
    in_channels=3,
    out_channels=1,
    kernel_size=3,
    stride=1,
    padding=0,
    dilation=2,
    groups=1,
    bias=True,
    padding_mode='zeros',
    )
torchFunction2 = torch.nn.Conv3d(
    in_channels=3,
    out_channels=1,
    kernel_size=3,
    stride=1,
    padding=0,
    dilation=2,
    groups=1,
    bias=True,
    padding_mode='zeros',
    )
torchFunction2.weight.data = kernel2
torchFunction2.bias.data = bias_val2
myResult = str(np.round(result2, 2))
torchResult = str(np.round(np.array(torchFunction2(tensor2).data), 2))
torchResult == myResult

True

In [13]:
result3, kernel3, bias_val3 = myConv3d(
    in_channels=1,
    out_channels=1,
    kernel_size=1,
    stride=1,
    padding=0,
    dilation=1,
    groups=1,
    bias=True,
    padding_mode='zeros',
    )
torchFunction3 = torch.nn.Conv3d(
    in_channels=1,
    out_channels=1,
    kernel_size=1,
    stride=1,
    padding=0,
    dilation=1,
    groups=1,
    bias=True,
    padding_mode='zeros',
    )
torchFunction3.weight.data = kernel3
torchFunction3.bias.data = bias_val3
myResult = str(np.round(result3, 2))
torchResult = str(np.round(np.array(torchFunction3(tensor3).data), 2))
torchResult == myResult

TypeError: myConv3d() missing 1 required positional argument: 'input_image'